<a href="https://colab.research.google.com/github/madhumitha-gv/NLP/blob/main/Spam_classifier_word2vec_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import pandas as pd
msg=pd.read_csv('/content/SMSSpamCollection.txt',sep='\t',names=['labels','message']) #data is tab seperated

In [17]:
msg

,labels,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [18]:
msg.shape

(5572, 2)

# Data Cleaning


In [19]:
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

#Stemming

In [20]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [21]:
corpus=[]
for i in range(0,len(msg)):
  review=re.sub('[^a-zA-Z0-9]',' ',msg['message'][i])
  review=review.lower()
  review=review.split()
  review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
  review=' '.join(review)
  corpus.append(review)

In [22]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [23]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [24]:
corpus=[]
for i in range(0,len(msg)):
  review=re.sub('[^a-zA-Z0-9]',' ',msg['message'][i])
  review=review.lower()
  review=review.split()

  review=[lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
  review=' '.join(review)
  corpus.append(review)

In [25]:
corpus

['go jurong point crazy available bugis n great world la e buffet cine got amore wat',
 'ok lar joking wif u oni',
 'free entry 2 wkly comp win fa cup final tkts 21st may 2005 text fa 87121 receive entry question std txt rate c apply 08452810075over18',
 'u dun say early hor u c already say',
 'nah think go usf life around though',
 'freemsg hey darling 3 week word back like fun still tb ok xxx std chgs send 1 50 rcv',
 'even brother like speak treat like aid patent',
 'per request melle melle oru minnaminunginte nurungu vettam set callertune caller press 9 copy friend callertune',
 'winner valued network customer selected receivea 900 prize reward claim call 09061701461 claim code kl341 valid 12 hour',
 'mobile 11 month u r entitled update latest colour mobile camera free call mobile update co free 08002986030',
 'gonna home soon want talk stuff anymore tonight k cried enough today',
 'six chance win cash 100 20 000 pound txt csh11 send 87575 cost 150p day 6days 16 tsandcs apply reply

In [26]:
corpus

['go jurong point crazy available bugis n great world la e buffet cine got amore wat',
 'ok lar joking wif u oni',
 'free entry 2 wkly comp win fa cup final tkts 21st may 2005 text fa 87121 receive entry question std txt rate c apply 08452810075over18',
 'u dun say early hor u c already say',
 'nah think go usf life around though',
 'freemsg hey darling 3 week word back like fun still tb ok xxx std chgs send 1 50 rcv',
 'even brother like speak treat like aid patent',
 'per request melle melle oru minnaminunginte nurungu vettam set callertune caller press 9 copy friend callertune',
 'winner valued network customer selected receivea 900 prize reward claim call 09061701461 claim code kl341 valid 12 hour',
 'mobile 11 month u r entitled update latest colour mobile camera free call mobile update co free 08002986030',
 'gonna home soon want talk stuff anymore tonight k cried enough today',
 'six chance win cash 100 20 000 pound txt csh11 send 87575 cost 150p day 6days 16 tsandcs apply reply

#Bag of Words Model

In [27]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=2500, binary=True,ngram_range=(2,2))
X=cv.fit_transform(corpus).toarray()


In [28]:
y=pd.get_dummies(msg['labels'])
y=y.iloc[:,1].values

In [29]:
X.shape

(5572, 2500)

In [30]:
y

array([False, False,  True, ..., False, False, False])

In [31]:
y.shape

(5572,)

In [32]:
y=pd.get_dummies(msg['labels'])

In [33]:
y=y.iloc[:,1].values

In [34]:
y.shape

(5572,)

## TrainTestSplit

In [35]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [36]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model=MultinomialNB().fit(X_train,y_train)

In [37]:
y_prediction=spam_detect_model.predict(X_test)

In [38]:
from sklearn.metrics import confusion_matrix
confusion_m=confusion_matrix(y_test,y_prediction)

In [39]:
confusion_m

array([[955,   0],
       [ 31, 129]])

In [40]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_test,y_prediction)
print(accuracy)

0.9721973094170404


In [41]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_prediction))

              precision    recall  f1-score   support

       False       0.97      1.00      0.98       955
        True       1.00      0.81      0.89       160

    accuracy                           0.97      1115
   macro avg       0.98      0.90      0.94      1115
weighted avg       0.97      0.97      0.97      1115



# creating TFIDF model

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv=TfidfVectorizer(max_features=2500,ngram_range=(1,2))
X=tv.fit_transform(corpus).toarray()

In [43]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [44]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model=MultinomialNB().fit(X_train,y_train)

In [45]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_prediction))

              precision    recall  f1-score   support

       False       0.97      1.00      0.98       955
        True       1.00      0.81      0.89       160

    accuracy                           0.97      1115
   macro avg       0.98      0.90      0.94      1115
weighted avg       0.97      0.97      0.97      1115



In [46]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_test,y_prediction)
print(accuracy)

0.9721973094170404


## word2vec

In [47]:
!pip install gensim

In [48]:
from nltk import sent_tokenize
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

In [49]:
import nltk
from nltk.tokenize import sent_tokenize
from gensim.utils import simple_preprocess

nltk.download('punkt_tab')  # Download tokenizer data (only needed once)

def tokenize_text(corpus):
    """
    Tokenizes text data into lists of preprocessed words.

    Args:
        corpus (list or str): A single text (string) or list of texts.

    Returns:
        list: A list of lists, where each inner list contains tokens for a sentence.
    """
    words = []

    # If input is a single string, wrap it as a list
    if isinstance(corpus, str):
        corpus = [corpus]

    for text in corpus:
        for sent in sent_tokenize(text):  # Sentence tokenization
            tokens = simple_preprocess(sent)  # Word tokenization & preprocessing
            words.append(tokens)

    return words


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [51]:
words=tokenize_text(corpus)

In [52]:
import gensim
model=gensim.models.Word2Vec(words, window=5, min_count=2)

In [53]:
model.corpus_count

5565

In [54]:
model.epochs

5

In [55]:
model.wv.similar_by_word('prize')

[('claim', 0.9995166659355164),
 ('call', 0.9992716908454895),
 ('line', 0.9992450475692749),
 ('cash', 0.9991818070411682),
 ('number', 0.9990391731262207),
 ('mobile', 0.9990181922912598),
 ('show', 0.9990110397338867),
 ('please', 0.9989802837371826),
 ('free', 0.9989765286445618),
 ('txt', 0.998971700668335)]

In [56]:
model.wv.index_to_key

['call',
 'get',
 'ur',
 'gt',
 'lt',
 'go',
 'ok',
 'day',
 'free',
 'know',
 'come',
 'like',
 'good',
 'time',
 'got',
 'text',
 'love',
 'want',
 'send',
 'need',
 'one',
 'txt',
 'today',
 'going',
 'stop',
 'home',
 'lor',
 'sorry',
 'see',
 'still',
 'take',
 'mobile',
 'back',
 'da',
 'reply',
 'dont',
 'think',
 'tell',
 'week',
 'phone',
 'hi',
 'new',
 'later',
 'please',
 'pls',
 'co',
 'msg',
 'dear',
 'make',
 'night',
 'message',
 'say',
 'well',
 'min',
 'thing',
 'much',
 'claim',
 'great',
 'hope',
 'oh',
 'hey',
 'give',
 'number',
 'happy',
 'work',
 'friend',
 'wat',
 'yes',
 'way',
 'www',
 'let',
 'prize',
 'right',
 'tomorrow',
 'already',
 'ask',
 'said',
 'win',
 'life',
 'amp',
 'cash',
 'yeah',
 'im',
 'really',
 'tone',
 'babe',
 'meet',
 'find',
 'miss',
 'morning',
 'service',
 'last',
 'uk',
 'thanks',
 'care',
 'would',
 'com',
 'anything',
 'year',
 'lol',
 'also',
 'nokia',
 'every',
 'feel',
 'keep',
 'sure',
 'pick',
 'sent',
 'urgent',
 'contact',


In [57]:
import numpy as np

In [58]:
def avg_word2vec(doc):
  return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key],axis=0)

In [59]:
!pip install tqdm

In [60]:
from tqdm import tqdm

In [61]:
type(model.wv.index_to_key)

list

In [62]:
type(X)

numpy.ndarray

In [63]:
X, y_new = [], []
num_features = model.wv.vector_size

# Filter out empty token lists from the words variable
words = [token_list for token_list in words if token_list]

for tokens, label in zip(words, y):
    vec = avg_word2vec(tokens)
    if vec is not None and vec.shape == (num_features,):
        X.append(vec)
        y_new.append(label)

X = np.vstack(X)  # Stack into 2D array safely
y = np.array(y_new)

/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.11/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [64]:
type(X)

numpy.ndarray

In [65]:
X

array([[-0.15096126,  0.3112108 ,  0.0829218 , ..., -0.25249216,
         0.04461214, -0.0435379 ],
       [-0.100868  ,  0.2055954 ,  0.05341213, ..., -0.16623685,
         0.02912683, -0.0250698 ],
       [-0.14194533,  0.28699088,  0.07878748, ..., -0.23496655,
         0.04451253, -0.03837245],
       ...,
       [-0.02999016,  0.0664976 ,  0.01967146, ..., -0.0484418 ,
         0.01583167, -0.01281164],
       [-0.1718118 ,  0.3452331 ,  0.09446228, ..., -0.28394705,
         0.05059481, -0.04990558],
       [-0.12131103,  0.2368667 ,  0.06887547, ..., -0.19759719,
         0.03067742, -0.03323868]], dtype=float32)

In [66]:
words[0]

['go',
 'jurong',
 'point',
 'crazy',
 'available',
 'bugis',
 'great',
 'world',
 'la',
 'buffet',
 'cine',
 'got',
 'amore',
 'wat']

In [67]:
words[1]

['ok', 'lar', 'joking', 'wif', 'oni']

In [68]:
words

[['go',
  'jurong',
  'point',
  'crazy',
  'available',
  'bugis',
  'great',
  'world',
  'la',
  'buffet',
  'cine',
  'got',
  'amore',
  'wat'],
 ['ok', 'lar', 'joking', 'wif', 'oni'],
 ['free',
  'entry',
  'wkly',
  'comp',
  'win',
  'fa',
  'cup',
  'final',
  'tkts',
  'st',
  'may',
  'text',
  'fa',
  'receive',
  'entry',
  'question',
  'std',
  'txt',
  'rate',
  'apply',
  'over'],
 ['dun', 'say', 'early', 'hor', 'already', 'say'],
 ['nah', 'think', 'go', 'usf', 'life', 'around', 'though'],
 ['freemsg',
  'hey',
  'darling',
  'week',
  'word',
  'back',
  'like',
  'fun',
  'still',
  'tb',
  'ok',
  'xxx',
  'std',
  'chgs',
  'send',
  'rcv'],
 ['even', 'brother', 'like', 'speak', 'treat', 'like', 'aid', 'patent'],
 ['per',
  'request',
  'melle',
  'melle',
  'oru',
  'minnaminunginte',
  'nurungu',
  'vettam',
  'set',
  'callertune',
  'caller',
  'press',
  'copy',
  'friend',
  'callertune'],
 ['winner',
  'valued',
  'network',
  'customer',
  'selected',
  're

In [69]:
X[0].shape

(100,)

In [70]:
y

array([False, False,  True, ..., False, False, False])

In [76]:
y.shape

(5541,)

In [77]:
X.shape

(5541, 100)

In [78]:
y = y.astype(int)

In [79]:
len_X = len(X)
len_y = len(y)
print(f"X has {len_X} samples, y has {len_y} samples")

X has 5541 samples, y has 5541 samples


In [80]:
import numpy as np
y = np.array(y, dtype=int)

In [81]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [82]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

log_reg = LogisticRegression(max_iter=1000, random_state=42)
log_reg.fit(X_train, y_train)

y_pred = log_reg.predict(X_test)

print("Baseline Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Baseline Accuracy: 0.8719567177637512

Classification Report:
               precision    recall  f1-score   support

           0       0.87      1.00      0.93       967
           1       0.00      0.00      0.00       142

    accuracy                           0.87      1109
   macro avg       0.44      0.50      0.47      1109
weighted avg       0.76      0.87      0.81      1109


Confusion Matrix:
 [[967   0]
 [142   0]]


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [83]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

params = {
    "n_estimators": [100, 200, 300],
    "max_depth": [None, 10, 20],
    "min_samples_split": [2, 5]
}

rf = RandomForestClassifier(random_state=42)

grid_rf = GridSearchCV(rf, params, cv=3, scoring='accuracy', n_jobs=-1, verbose=1)
grid_rf.fit(X_train, y_train)

best_rf = grid_rf.best_estimator_
print("Best Parameters:", grid_rf.best_params_)

y_pred_rf = best_rf.predict(X_test)

print("Tuned RF Accuracy:", accuracy_score(y_test, y_pred_rf))
print("\nClassification Report:\n", classification_report(y_test, y_pred_rf))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred_rf))


Fitting 3 folds for each of 18 candidates, totalling 54 fits
Best Parameters: {'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 100}
Tuned RF Accuracy: 0.8710550045085663

Classification Report:
               precision    recall  f1-score   support

           0       0.87      1.00      0.93       967
           1       0.00      0.00      0.00       142

    accuracy                           0.87      1109
   macro avg       0.44      0.50      0.47      1109
weighted avg       0.76      0.87      0.81      1109


Confusion Matrix:
 [[966   1]
 [142   0]]


In [84]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.1,
    eval_metric="logloss",
    random_state=42
)

xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

print("XGBoost Accuracy:", accuracy_score(y_test, y_pred_xgb))
print("\nClassification Report:\n", classification_report(y_test, y_pred_xgb))


XGBoost Accuracy: 0.8539224526600541

Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.98      0.92       967
           1       0.00      0.00      0.00       142

    accuracy                           0.85      1109
   macro avg       0.43      0.49      0.46      1109
weighted avg       0.76      0.85      0.80      1109



In [85]:
import joblib

joblib.dump(best_rf, "spam_classifier_rf.pkl")
print("Model saved as spam_classifier_rf.pkl")

Model saved as spam_classifier_rf.pkl


In [86]:
def predict_spam(message, model, w2v_model):
    # Get tokens from the message
    tokens_list = tokenize_text(message)  # returns a list of token lists

    # Merge all tokens (since message may have multiple sentences)
    tokens = [word for sent in tokens_list for word in sent]

    # Compute average Word2Vec vector
    vec = avg_word2vec(tokens)

    if vec is None:
        return "Cannot classify (no known words in message)"

    # Reshape to 2D array for prediction
    vec = vec.reshape(1, -1)

    prediction = model.predict(vec)[0]

    return "Spam" if prediction == 1 else "Not Spam"


In [87]:
msg = "Congratulations! You have won a $500 gift card. Click here to claim now!"
print(predict_spam(msg, best_rf, model))

Not Spam
